In [1]:
from hypex.dataset import Dataset, ExperimentData, InfoRole, TreatmentRole, TargetRole
from hypex.experiments.aa import ONE_AA_TEST
from hypex.reporters import AADictReporter

C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = Dataset(
    roles={
        "user_id": InfoRole(float),
        "treat": TreatmentRole(int),
        "pre_spends": TargetRole(),
        "post_spends": TargetRole()
    }, data="data.csv",
)
data

      user_id  signup_month  treat  pre_spends  post_spends   age gender  \
0         0.0             0      0       488.0   414.444444   NaN      M   
1         1.0             8      1       512.5   462.222222  26.0    NaN   
2         2.0             7      1       483.0   479.444444  25.0      M   
3         3.0             0      0       501.5   424.333333  39.0      M   
4         4.0             1      1       543.0   514.555556  18.0      F   
...       ...           ...    ...         ...          ...   ...    ...   
9995   9995.0            10      1       538.5   450.444444  42.0      M   
9996   9996.0             0      0       500.5   430.888889  26.0      F   
9997   9997.0             3      1       473.0   534.111111  22.0      F   
9998   9998.0             2      1       495.0   523.222222  67.0      F   
9999   9999.0             7      1       508.0   475.888889  38.0      F   

        industry  
0     E-commerce  
1     E-commerce  
2      Logistics  
3     E-com

In [3]:
data.data.dtypes

user_id          float64
signup_month       int32
treat              int32
pre_spends       float64
post_spends      float64
age              float64
gender            object
industry        category
dtype: object

In [4]:
test = ONE_AA_TEST
ed = ExperimentData(data)
result = test.execute(ed)

In [5]:
result.analysis_tables

{'GroupSizes┴┴':    control size  test size  control size %  test size %
 0          5000       5000            50.0         50.0,
 'GroupDifference┴┴pre_spends':    control mean  test mean  difference  difference %
 0      487.0277   487.1598      0.1321      0.027124,
 'TTest┴┴pre_spends':     p-value  statistic   pass
 0  0.726345  -0.350001  False,
 'KSTest┴┴pre_spends':     p-value  statistic   pass
 0  0.677788     0.0144  False,
 'GroupDifference┴┴post_spends':    control mean   test mean  difference  difference %
 0      452.4788  451.850311   -0.628489     -0.138899,
 'TTest┴┴post_spends':     p-value  statistic   pass
 0  0.425194   0.797475  False,
 'KSTest┴┴post_spends':     p-value  statistic   pass
 0  0.627218      0.015  False,
 'OneAAStatAnalyzer┴┴':    mean TTest p-value  mean TTest pass  mean KSTest p-value  mean KSTest pass  \
 0             0.57577              0.0             0.652503               0.0   
 
    mean test score  
 0         0.626925  }

In [6]:
AADictReporter().report(result)

{'splitter_id': 'AASplitter┴None┴',
 'pre_spends GroupDifference control mean 0': 487.0277,
 'pre_spends GroupDifference test mean 0': 487.1598,
 'pre_spends GroupDifference difference 0': 0.13210000000003674,
 'pre_spends GroupDifference difference % 0': 0.027123713907850622,
 'post_spends GroupDifference control mean 0': 452.4788,
 'post_spends GroupDifference test mean 0': 451.8503111111112,
 'post_spends GroupDifference difference 0': -0.6284888888887963,
 'post_spends GroupDifference difference % 0': -0.13889907966710968,
 'GroupSizes control size 0': 5000,
 'GroupSizes test size 0': 5000,
 'GroupSizes control size % 0': 50.0,
 'GroupSizes test size % 0': 50.0,
 'pre_spends TTest p-value 0': 0.726345178714652,
 'pre_spends TTest pass 0': 'OK',
 'post_spends TTest p-value 0': 0.4251942678027748,
 'post_spends TTest pass 0': 'OK',
 'pre_spends KSTest p-value 0': 0.6777877521935483,
 'pre_spends KSTest pass 0': 'OK',
 'post_spends KSTest p-value 0': 0.6272175062405767,
 'post_spends 